# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 47 new papers today
          25 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/24 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2211.05131


extracting tarball to tmp_2211.05131...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.05134


extracting tarball to tmp_2211.05134...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.05754


extracting tarball to tmp_2211.05754... done.
Retrieving document from  https://arxiv.org/e-print/2211.05779


extracting tarball to tmp_2211.05779...

 done.


/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3433: LatexWarning: Found documentclass in tmp_2211.05779/MGLenS.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.15/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'abbrev' from 'tmp_2211.05779/abbrev.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.05134-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.05134) | **SN 2022ann: A type Icn supernova from a dwarf galaxy that reveals helium  in its circumstellar environment**  |
|| K. W. Davis, et al. -- incl., <mark>V. A. Villar</mark>, <mark>M. E. Huber</mark>, <mark>D. Langeroodi</mark>, <mark>F. R. Herpich</mark>, <mark>C.-C. Lin</mark> |
|*Appeared on*| *2022-11-11*|
|*Comments*| *21 pages, 13 figures, to be submitted to MNRAS*|
|**Abstract**| We present optical and near-infrared (NIR) observations of the Type Icn supernova (SN Icn) 2022ann, the fifth member of its newly identified class of SNe. Its early optical spectra are dominated by narrow carbon and oxygen P-Cygni features with absorption velocities of 800 km/s; slower than other SNe Icn and indicative of interaction with a dense, H/He-poor circumstellar medium (CSM) that is outflowing slower than a typical Wolf-Rayet wind velocity of $>$1000 km/s. We identify helium in NIR spectra obtained two weeks after maximum and in optical spectra at three weeks, demonstrating that the CSM is not fully devoid of helium. We never detect broad spectral features from SN ejecta, including in spectra extending to the nebular phase, a unique characteristic among SNe~Icn. Compared to other SNe Icn, SN 2022ann has a low luminosity, with a peak o-band absolute magnitude of -17.7, and evolves slowly. We model the bolometric light curve and find it is well-described by 1.7 M_Sun of SN ejecta interacting with 0.2 M_sun of CSM. We place an upper limit of 0.04 M_Sun of Ni56 synthesized in the explosion. The host galaxy is a dwarf galaxy with a stellar mass of 10^7.34 M_Sun (implied metallicity of log(Z/Z_Sun) $\approx$ 0.10) and integrated star-formation rate of log(SFR) = -2.20 M_sun/yr; both lower than 97\% of the galaxies observed to produce core-collapse supernovae, although consistent with star-forming galaxies on the galaxy Main Sequence. The low CSM velocity, nickel and ejecta masses, and likely low-metallicity environment disfavour a single Wolf-Rayet progenitor star. Instead, a binary companion star is likely required to adequately strip the progenitor before explosion and produce a low-velocity outflow. The low CSM velocity may be indicative of the outer Lagrangian points in the stellar binary progenitor, rather than from the escape velocity of a single Wolf-Rayet-like massive star. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.05131-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.05131) | **Toward Horizon-scale Accretion Onto Supermassive Black Holes in  Elliptical Galaxies**  |
|| <mark>Minghao Guo</mark>, James M. Stone, Chang-Goo Kim, Eliot Quataert |
|*Appeared on*| *2022-11-11*|
|*Comments*| *28 pages, 24 figures, 2 tables, submitted to ApJ, comments are welcome*|
|**Abstract**| We present high-resolution, three-dimensional hydrodynamic simulations of the fueling of supermassive black holes in elliptical galaxies from a turbulent medium on galactic scales, taking M87* as a typical case. The simulations use a new GPU-accelerated version of the Athena++ AMR code, and span more than 6 orders of magnitude in radius, reaching scales similar to the black hole horizon. The key physical ingredients are radiative cooling and a phenomenological heating model. We find that the accretion flow takes the form of multiphase gas at radii less than about a kpc. The cold gas accretion includes two dynamically distinct stages: the typical disk stage in which the cold gas resides in a rotationally supported disk and relatively rare chaotic stages ($\lesssim 10\%$ of the time) in which the cold gas inflows via chaotic streams. Though cold gas accretion dominates the time-averaged accretion rate at intermediate radii, accretion at the smallest radii is dominated by hot virialized gas at most times. The accretion rate scales with radius as $\dot{M}\propto r^{1/2}$ when hot gas dominates and we obtain $\dot{M}\simeq10^\mathrm{-4}-10^\mathrm{-3}\,M_\odot\,\mathrm{yr^{-1}}$ near the event horizon, similar to what is inferred from EHT observations. The orientation of the cold gas disk can differ significantly on different spatial scales. We propose a subgrid model for accretion in lower-resolution simulations in which the hot gas accretion rate is suppressed relative to the Bondi rate by $\sim (r_\mathrm{g}/r_{\rm Bondi})^{1/2}$. Our results can also provide more realistic initial conditions for simulations of black hole accretion at the event horizon scale. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.05141-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.05141) | **Volumetric rates of Luminous Red Novae and Intermediate Luminosity Red  Transients with the Zwicky Transient Facility**  |
|| Viraj R. Karambelkar, et al. -- incl., <mark>George Helou</mark>, <mark>Frank J. Masci</mark>, <mark>Yashvi Sharma</mark>, <mark>Chaoran Zhang</mark> |
|*Appeared on*| *2022-11-11*|
|*Comments*| *32 pages, 16 figures. Submitted to ApJ, comments welcome*|
|**Abstract**| Luminous red novae (LRNe) are transients characterized by low luminosities and expansion velocities, and are associated with mergers or common envelope ejections in stellar binaries. Intermediate-luminosity red transients (ILRTs) are an observationally similar class with unknown origins, but generally believed to either be electron capture supernovae (ECSN) in super-AGB stars, or outbursts in dusty luminous blue variables (LBVs). In this paper, we present a systematic sample of 8 LRNe and 8 ILRTs detected as part of the Census of the Local Universe (CLU) experiment on the Zwicky Transient Facility (ZTF). The CLU experiment spectroscopically classifies ZTF transients associated with nearby ($<150$ Mpc) galaxies, achieving 80% completeness for m$_{r}<20$\,mag. Using the ZTF-CLU sample, we derive the first systematic LRNe volumetric-rate of 7.8$^{+6.5}_{-3.7}\times10^{-5}$ Mpc$^{-3}$ yr$^{-1}$ in the luminosity range $-16\leq$M$_{\rm{r}}$$\leq -11$ mag. We find that in this luminosity range, the LRN rate scales as dN/dL $\propto L^{-2.5\pm0.3}$ - significantly steeper than the previously derived scaling of $L^{-1.4\pm0.3}$ for lower luminosity LRNe (M$_{V}\geq-10$). The steeper power law for LRNe at high luminosities is consistent with the massive merger rates predicted by binary population synthesis models. We find that the rates of the brightest LRNe (M$_{r}\leq-13$ mag) are consistent with a significant fraction of them being progenitors of double compact objects (DCOs) that merge within a Hubble time. For ILRTs, we derive a volumetric rate of $2.6^{+1.8}_{-1.4}\times10^{-6}$ Mpc$^{-3}$yr$^{-1}$ for M$_{\rm{r}}\leq-13.5$, that scales as dN/dL $\propto L^{-2.5\pm0.5}$. This rate is $\approx1-5\%$ of the local core-collapse supernova rate, and is consistent with theoretical ECSN rate estimates. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.05744-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.05744) | **Identification of Large Equivalent Width Dusty Galaxies at 4 $<$ z $<$ 6  from Sub-mm Colours**  |
|| Denis Burgarella, et al. -- incl., <mark>Lisa Gouiran</mark> |
|*Appeared on*| *2022-11-11*|
|*Comments*| *Paper accepted in Astronomy and Astrophysics on 10 November 2022*|
|**Abstract**| Infrared (IR), sub-millimetre (sub-mm) and millimetre (mm) databases contain a huge quantity of high quality data. However, a large part of these data are photometric, and are thought not to be useful to derive a quantitative information on the nebular emission of galaxies. The aim of this project is first to identify galaxies at z > 4-6, and in the epoch of reionization from their sub-mm colours. We also aim at showing that the colours can be used to try and derive physical constraints from photometric bands, when accounting for the contribution from the IR fine structure lines to these photometric bands. We model the flux of IR fine structure lines with CLOUDY, and add them to the dust continuum emission with CIGALE. Including or not emission lines in the simulated spectral energy distribution (SED) modifies the broad band emission and colours. The introduction of the lines allows to identify strong star forming galaxies at z > 4 - 6 from the log10 (PSW_250um/PMW_350um) versus log10 (LABOCA_870um/PLW_500um) colour-colour diagramme. By comparing the relevant models to each observed galaxy colour, we are able to roughly estimate the fluxes of the lines, and the associated nebular parameters. This method allows to identify a double sequence in a plot built from the ionization parameter and the gas metallicity. The HII and photodissociation region (PDR) fine structure lines are an essential part of the SEDs. It is important to add them when modelling the spectra, especially at z > 4 - 6 where their equivalent widths can be large. Conversely, we show that we can extract some information on strong IR fine structure lines and on the physical parameters related to the nebular emission from IR colour-colour diagrams. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.05754-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.05754) | **External Inverse-Compton and Proton Synchrotron Emission from the  Reverse Shock as the Origin of VHE Gamma-Rays from the Hyper-Bright GRB  221009A**  |
|| <mark>B. Theodore Zhang</mark>, et al. |
|*Appeared on*| *2022-11-11*|
|*Comments*| *9 pages, 4 figures*|
|**Abstract**| The detection of the hyper-bright gamma-ray burst (GRB) 221009A enables us to explore the nature of GRB emission and the origin of very-high-energy (VHE) gamma-rays. We analyze the ${\it Fermi}$-LAT data and investigate GeV-TeV emission in the framework of the external reverse shock model. We show that early $\sim1-10$ GeV emission can be explained by the external inverse-Compton mechanism via upscattering MeV gamma-rays by electrons accelerated at the reverse shock, in addition to the synchrotron self-Compton component. The predicted early optical flux could have been brighter than the naked-eye GRB 080319B. We also show that proton synchrotron emission from accelerated ultra-high-energy cosmic rays (UHECRs) is detectable, and could potentially explain $\gtrsim \rm TeV$ photons detected by LHAASO or UHECR acceleration can be constrained. Our model suggests that the detection of $\mathcal{O}(10\rm~TeV)$ photons with energy up to $\sim18$ TeV is possible for reasonable models of the extragalactic background light without invoking new physics, and predicts anti-correlations between MeV photons and TeV photons, which can be tested with the LHAASO data |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.05779-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.05779) | **MGLenS: Modified gravity weak lensing simulations for emulation-based  cosmological inference**  |
|| Joachim Harnois-Déraps, et al. -- incl., <mark>Cesar Hernandez-Aguayo</mark>, <mark>Baojiu Li</mark>, <mark>Christopher T. Davies</mark> |
|*Appeared on*| *2022-11-11*|
|*Comments*| *22 pages, 15 figures, comments welcome, simulations available upon request*|
|**Abstract**| We present MGLenS, a large series of modified gravity lensing simulations tailored for cosmic shear data analyses and forecasts in which cosmological and modified gravity parameters are varied simultaneously. Based on the FORGE and BRIDGE $N$-body simulation suites presented in companion papers, we construct 500,000 deg$^2$ of mock Stage-IV lensing data, sampling a pair of 4-dimensional volumes designed for the training of emulators. We validate the accuracy of MGLenS with inference analyses based on the lensing power spectrum exploiting our implementation of $f(R)$ and nDGP theoretical predictions within the cosmoSIS cosmological inference package. A Fisher analysis reveals that the vast majority of the constraining power from such a survey comes from the highest redshift galaxies alone. We further find from a full likelihood sampling that cosmic shear can achieve 95% CL constraints on the modified gravity parameters of log$_{10}\left[ f_{R_0}\right] < -5.24$ and log$_{10}\left[ H_0 r_c\right] > -0.05$, after marginalising over intrinsic alignments of galaxies and including scales up to $\ell=5000$. Such a survey setup could in fact detect with more than $3\sigma$ confidence $f(R)$ values larger than $3 \times 10^{-6}$ and $H_0 r_c$ smaller than 1.0. Scale cuts at $\ell=3000$ reduce the degeneracy breaking between $S_8$ and the modified gravity parameters, while photometric redshift uncertainty seem to play a subdominant role in our error budget. We finally explore the consequences of analysing data with the wrong gravity model, and report the catastrophic biases for a number of possible scenarios. The Stage-IV MGLenS simulations, the FORGE and BRIDGE emulators and the cosmoSIS interface modules will be made publicly available upon journal acceptance. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.05573-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.05573) | **The Star Formation Rate of the Milky Way as seen by Herschel**  |
|| D. Elia, et al. -- incl., <mark>P. Hennebelle</mark>, <mark>S. J. Liu</mark> |
|*Appeared on*| *2022-11-11*|
|*Comments*| *To be published in ApJ*|
|**Abstract**| We present a new derivation of the Milky Way's current star formation rate (SFR) based on the data of the Hi-GAL Galactic plane survey. We estimate the distribution of the SFR across the Galactic plane from the star-forming clumps identified in the Hi-GAL survey and calculate the total SFR from the sum of their contributions. The estimate of the global SFR amounts to $2.0 \pm 0.7$~M$_{\odot}$~yr$^{-1}$, of which $1.7 \pm 0.6$~M$_{\odot}$~yr$^{-1}$ coming from clumps with reliable heliocentric distance assignment. This value is in general agreement with estimates found in the literature of last decades. The profile of SFR density averaged in Galactocentric rings is found to be qualitatively similar to others previously computed, with a peak corresponding to the Central Molecular Zone and another one around Galactocentric radius $R_\mathrm{gal} \sim 5$~kpc, followed by an exponential decrease as $\log(\Sigma_\mathrm{SFR}/[\mathrm{M}_\odot~\mathrm{yr}^{-1}~\mathrm{kpc}^{-2}])=a\,R_\mathrm{gal}/[\mathrm{kpc}]+b $, with $a=-0.28 \pm 0.01$. In this regard, the fraction of SFR produced within and outside the Solar circle is 84\% and 16\%, respectively; the fraction corresponding to the far outer Galaxy ($R_\mathrm{gal} > 13.5$~kpc) is only 1\%. We also find that, for $R_\mathrm{gal}>3$~kpc, our data follow a power law as a function of density, similarly to the Kennicutt-Schmidt relation. Finally, we compare the distribution of the SFR density across the face-on Galactic plane and those of median parameters, such as temperature, luminosity/mass ratio and bolometric temperature, describing the evolutionary stage of Hi-GAL clumps. We found no clear correlation between the SFR and the clump evolutionary stage. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.05155-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.05155) | **Signatures of Strong Magnetization and Metal-Poor Atmosphere for a  Neptune-Size Exoplanet**  |
|| Lotfi Ben-Jaffel, et al. -- incl., <mark>Gregory W. Henry</mark>, <mark>Thomas Mikal-Evans</mark> |
|*Appeared on*| *2022-11-11*|
|*Comments*| *68 pages, 12 figures. Published in Nature Astronomy on December 16 2021. Main draft and Supplementary information are included in a single file. Full-text access to a view-only version of the paper via : this https URL*|
|**Abstract**| The magnetosphere of an exoplanet has yet to be unambiguously detected. Investigations of star-planet interaction and neutral atomic hydrogen absorption during transit to detect magnetic fields in hot Jupiters have been inconclusive, and interpretations of the transit absorption non-unique. In contrast, ionized species escaping a magnetized exoplanet, particularly from the polar caps, should populate the magnetosphere, allowing detection of different regions from the plasmasphere to the extended magnetotail, and characterization of the magnetic field producing them. Here, we report ultraviolet observations of HAT-P-11b, a low-mass (0.08 MJ) exoplanet showing strong, phase-extended transit absorption of neutral hydrogen (maximum and tail transit depths of 32 \pm 4%, 27 \pm 4%) and singly ionized carbon (15 \pm 4%, 12.5 \pm 4%). We show that the atmosphere should have less than six times the solar metallicity (at 200 bars), and the exoplanet must also have an extended magnetotail (1.8-3.1 AU). The HAT-P-11b equatorial magnetic field strength should be about 1-5 Gauss. Our panchromatic approach using ionized species to simultaneously derive metallicity and magnetic field strength can now constrain interior and dynamo models of exoplanets, with implications for formation and evolution scenarios. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

85  publications in the last 7 days.
	 _build/html/2211.05022.md
	 _build/html/2211.04048.md
	 _build/html/2211.03641.md
	 _build/html/2211.02502.md
	 _build/html/2211.01474.md
	 _build/html/2211.00667.md
	 _build/html/2211.00657.md
	 _build/html/2211.00087.md
	 _build/html/2211.00036.md
	 _build/html/2211.00020.md
	 _build/html/2211.00010.md
	 _build/html/2210.17434.md
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers